This notebook is used to turn the collected data into something more useable and consumable. I turned what I collected into various different statistical catagories that will help my model become more accurate at prediction. This could not have been possible without using K1NG's Overwatch League Spreadsheet. You can find it here:
https://docs.google.com/spreadsheets/d/1KJGwLRMDJP1SSakt25laZbDaTNYRHZYHJmV6rh86jnc/edit?usp=sharing

Note: This will need to constantly be updated as more maps and gametypes get added. Order of maps is very important (as evidenced in cell 4)


In [1]:
#Getting proper libraries
import pandas as pd
import numpy as np
import re

In [2]:
#Getting Dictionary of Team Names
from constants import ID_TO_NAME

In [3]:
#This gets the 2019 data. Will change this later to be more flexable to allow any year data.
df= pd.read_csv("data2019.csv")


teamnames = [7698, 4402, 7692, 4523, 4407, 7699, 7693, 4525, 4410, 4406, 4405, 4403, 7694, 4524, 4404, 4409, 4408, 7695, 7696, 7697]
team_series = []

#Filtering Dataframe to numerical data
colums = ["away", "away id", "away score", "home", "home id", "home score", "winner", 
            "Map 1 Name", "Map 1 Type", "Map 1 Away Points", "Map 1 Home Points", "Map 1 Winner", 
            "Map 2 Name", "Map 2 Type", "Map 2 Away Points", "Map 2 Home Points", "Map 2 Winner", 
            "Map 3 Name", "Map 3 Type", "Map 3 Away Points", "Map 3 Home Points", "Map 3 Winner", 
            "Map 4 Name", "Map 4 Type", "Map 4 Away Points", "Map 4 Home Points", "Map 4 Winner",
            "Map 5 Name", "Map 5 Type", "Map 5 Away Points", "Map 5 Home Points", "Map 5 Winner"]

df = df[colums]

#Columns that will be recorded
map_colums = ["Wins" , "Losses", "Draws", "Times Played", "Average Points Earned", 
              "Average Points Lost", "Average Points Differential", "Win %", "True %", "Map Potential %"]

#iterates through each team and collects all map statistics
for i in teamnames:
    control = []
    assault = []
    hybrid = []
    escort = []
    teamdf = df[df.eq(i).any(1)]
    for index, row in teamdf.iterrows():
        home_team = row["home"]
        away_team = row["away"]
        away_id = row["away id"]
        if(away_id != i):
            opponent = away_team
            weare = "Home"
        else:
            opponent = home_team
            weare = "Away"
        opp = pd.Series([opponent, weare], index =["Opponent", "Home or Away"])
        for column in row.iteritems():
            if column[0].find("Type") > -1:
                map_name = column[0].replace(" Type", " ")
                if isinstance(column[1], str):
                    index = ["Name", "Type", "Away Points", "Home Points", "Winner"]
                    if column[1].find("Control") > -1:
                        data = row[(map_name + "Name"):(map_name + "Winner")]
                        data.index = index
                        data = pd.concat([data, opp])
                        control.append(data)
                    elif column[1].find("Assault") > -1:
                        data = row[(map_name + "Name"):(map_name + "Winner")]
                        data.index = index
                        data = pd.concat([data, opp])
                        assault.append(data)
                    elif column[1].find("Hybrid") > -1:
                        data = row[(map_name + "Name"):(map_name + "Winner")]
                        data.index = index
                        data = pd.concat([data, opp])
                        hybrid.append(data)
                    elif column[1].find("Escort") > -1:
                        data = row[(map_name + "Name"):(map_name + "Winner")]
                        data.index = index
                        data = pd.concat([data, opp])
                        escort.append(data)
                    else:
                        continue
    #Assault
    hanamura = []
    horizon = []
    paris = []
    temple = []
    volskaya = []
    
    #Control
    busan = []
    ilios = []
    lijiang = []
    nepal = []
    oasis = []
    
    #Hybrid
    blizzard = []
    eichenwalde = []
    hollywood = []
    king = []
    numbani = []
    
    #Escort
    dorado = []
    havana = []
    junkertown = []
    rialto = []
    route = []
    gibraltar = []
    
    def get_stats(thelist):
        name = thelist[0].Name
        loss = round(0, 3)
        win = round(0, 3)
        draw = round(0, 3)
        points_earned = round(0, 3)
        points_given = round(0, 3)
        map_potential = round(0, 3)

        total = len(thelist)
        for game in thelist:
            
            if game.Winner != game["Home or Away"]:
                if game.Winner == "Draw":
                    draw += 1
                else:
                    loss += 1
            else:
                win += 1

            if game["Home or Away"] == "Home":
                points_earned += game["Home Points"]
                points_given += game["Away Points"]
                pwon = game["Home Points"]
                plos = game["Away Points"]

            else:
                points_earned += game["Away Points"]
                points_given += game["Home Points"]
                pwon = game["Away Points"]
                plos = game["Home Points"]

            if pwon > plos:
                map_potential += 1
            else:
                try:
                    map_potential += (pwon / plos)
                except ZeroDivisionError:
                    map_potential += 0

        win_per = round((win / total) * 100, 3)
        true_per = round(((win + (draw / 2)) / total) * 100, 3)
        avg_points_earned = round(points_earned / total, 3)
        avg_points_given = round(points_given / total, 3)
        avg_point_diff = round(avg_points_earned - avg_points_given, 3)
        true_map_potential = round(map_potential / total, 3)
        
        colums = [name + "_" + w for w in map_colums]

        map_data = pd.Series([win, loss, draw, total,
                              avg_points_earned, avg_points_given, avg_point_diff, win_per, true_per, 
                              true_map_potential], colums)
        return map_data

           
    for j in assault:
        if j.Name.find("hanamura") > -1:
            hanamura.append(j)
        elif j.Name.find("horizon-lunar-colony") > -1:
            horizon.append(j)
        elif j.Name.find("paris") > -1:
            paris.append(j)
        elif j.Name.find("temple-of-anubis") > -1:
            temple.append(j)
        elif j.Name.find("volskaya") > -1:
            volskaya.append(j)
        else:
            continue
    assault_data = pd.concat([get_stats(hanamura), 
                              get_stats(horizon), 
                              get_stats(paris),
                              get_stats(temple),
                              get_stats(volskaya)])
            
    for j in control:
        if j.Name.find("busan") > -1:
            busan.append(j)
        elif j.Name.find("ilios") > -1:
            ilios.append(j)
        elif j.Name.find("lijiang") > -1:
            lijiang.append(j)
        elif j.Name.find("nepal") > -1:
            nepal.append(j)
        elif j.Name.find("oasis") > -1:
            oasis.append(j)
        else:
            continue
    control_data = pd.concat([get_stats(busan),
                              get_stats(ilios), 
                              get_stats(lijiang), 
                              get_stats(nepal), 
                              get_stats(oasis)
                             ])
    
    for j in hybrid:
        if j.Name.find("blizzard-world") > -1:
            blizzard.append(j)
        elif j.Name.find("eichenwalde") > -1:
            eichenwalde.append(j)
        elif j.Name.find("hollywood") > -1:
            hollywood.append(j)
        elif j.Name.find("kings-row") > -1:
            king.append(j)
        elif j.Name.find("numbani") > -1:
            numbani.append(j)
        else:
            continue
    hybrid_data = pd.concat([get_stats(blizzard),
                             get_stats(eichenwalde),
                             get_stats(hollywood),
                             get_stats(king),
                             get_stats(numbani)])
    
    for j in escort:
        if j.Name.find("dorado") > -1:
            dorado.append(j)
        elif j.Name.find("havana") > -1:
            havana.append(j)
        elif j.Name.find("junkertown") > -1:
            junkertown.append(j)
        elif j.Name.find("rialto") > -1:
            rialto.append(j)
        elif j.Name.find("route-66") > -1:
            route.append(j)
        elif j.Name.find("gibraltar") > -1:
            gibraltar.append(j)
        else:
            continue
    escort_data = pd.concat([get_stats(dorado),
                             get_stats(havana),
                             get_stats(junkertown),
                             get_stats(rialto),
                             get_stats(route),
                             get_stats(gibraltar)])
    
    complete_data = pd.concat([assault_data, control_data, hybrid_data, escort_data])
    team_name = pd.Series([ID_TO_NAME.get(i)], index = ["Team Name"])
    complete_data = pd.concat([team_name, complete_data])
    team_series.append(complete_data)


In [4]:
#Turn data into dataframe
datadf = pd.DataFrame(team_series)

#Assign map names to each series of data
map_names = ["busan","ilios","lijiang","nepal","oasis",
             "hanamura","horizon-lunar-colony","paris","temple-of-anubis","volskaya",
             "blizzard-world","eichenwalde","hollywood","kings-row","numbani",
             "dorado","havana","junkertown","rialto","route-66","gibraltar"]

#Adds Percentages and Rankings for each team
for stringtest in map_names:
    wincol = datadf[stringtest+"_Win %"].rank(ascending = False, method = "min")
    truecol = datadf[stringtest+"_True %"].rank(ascending = False, method = "min")
    mappolcol = datadf[stringtest+"_Map Potential %"].rank(ascending = False, method = "min")
    datadf.insert(datadf.columns.get_loc(stringtest+"_Win %") + 1, stringtest+"_Win % Rank", wincol )
    datadf.insert(datadf.columns.get_loc(stringtest+"_True %") + 1, stringtest+"_True % Rank", truecol )
    datadf.insert(datadf.columns.get_loc(stringtest+"_Map Potential %") + 1, stringtest+"_Map Potential % Rank", mappolcol )
    

In [7]:
#renaming for writeability
stats= datadf

#clearing all previous items in list, as the list is reused
assault.clear()
control.clear()
hybrid.clear()
escort.clear()

#creating new specific lists
times_played = []
earned = []
lost = []
differ = []
winperc = []
trueperc = []
cows = []

#filtering maps based on map type
for i in stats.columns:
    if re.search("hanamura\B", i) or re.search("horizon-lunar-colony\B", i) or re.search("paris\B", i) or re.search("temple-of-anubis\B", i) or re.search("volskaya\B", i):
        assault.append(i)
    elif re.search("busan\B", i) or re.search("ilios\B", i) or re.search("lijiang\B", i) or re.search("nepal\B", i) or re.search("oasis\B", i):
        control.append(i)
    elif re.search("blizzard-world\B", i) or re.search("eichenwalde\B", i) or re.search("hollywood\B", i) or re.search("kings-row\B", i) or re.search("numbani\B", i):
        hybrid.append(i)
    elif re.search("dorado\B", i) or re.search("havana\B", i) or re.search("junkertown\B", i) or re.search("rialto\B", i) or re.search("route-66\B", i) or re.search("gibraltar\B", i):
        escort.append(i)
    else:
        continue

#this aggregates data across the map type, and returns statictics
def sum_rows(maptype, name): 
    times_played = [i for i in maptype if "Times Played" in i]
    times_playedtotal = stats[times_played].sum(axis = 1)
    
    earned = [i for i in maptype if "Average Points Earned" in i]
    earnedtotal = stats[earned].mean(axis = 1)
    
    lost = [i for i in maptype if "Average Points Lost" in i]
    losttotal = stats[lost].mean(axis = 1)
    
    differ = [i for i in maptype if "Average Points Differential" in i]
    differtotal = stats[differ].mean(axis = 1)
    sa = pd.DataFrame(data = differtotal)
    sa["total"] = differtotal
    differrank = sa["total"].rank(ascending = False, method = "min")
    
    winperc = [i for i in maptype if "Win %" in i and "Rank" not in i]
    winperctotal = stats[winperc].mean(axis = 1)
    sa["total"] = winperctotal
    winpercrank = sa["total"].rank(ascending = False, method = "min")
    
    trueperc = [i for i in maptype if "True %" in i and "Rank" not in i]
    trueperctotal = stats[trueperc].mean(axis = 1)
    sa["total"] = trueperctotal
    truepercrank = sa["total"].rank(ascending = False, method = "min")
    
    cows = [i for i in maptype if "Map Potential %" in i and "Rank" not in i]
    cowstotal = stats[cows].mean(axis = 1)
    sa["total"] = cowstotal
    cowsrank = sa["total"].rank(ascending = False, method = "min")
    
    if name == "Assault":
        stats.insert(stats.columns.get_loc("volskaya_Map Potential % Rank") + 1, "Total Assault Played", times_playedtotal)
        stats.insert(stats.columns.get_loc("Total Assault Played") + 1, "Average Assault Points Earned", earnedtotal)
        stats.insert(stats.columns.get_loc("Average Assault Points Earned") + 1, "Average Assault Points Lost", losttotal)
        stats.insert(stats.columns.get_loc("Average Assault Points Lost") + 1, "Average Assault Points Differential", differtotal)
        stats.insert(stats.columns.get_loc("Average Assault Points Differential") + 1, "Average Assault Points Differential Rank", differrank)
        stats.insert(stats.columns.get_loc("Average Assault Points Differential Rank") + 1, "Average Assault Win %", winperctotal)
        stats.insert(stats.columns.get_loc("Average Assault Win %") + 1, "Average Assault Win % Rank", winpercrank)
        stats.insert(stats.columns.get_loc("Average Assault Win % Rank") + 1, "Average Assault True Win %", trueperctotal)
        stats.insert(stats.columns.get_loc("Average Assault True Win %") + 1, "Average Assault True Win % Rank", truepercrank)
        stats.insert(stats.columns.get_loc("Average Assault True Win % Rank") + 1, "Average Assault Map Potential %", cowstotal)
        stats.insert(stats.columns.get_loc("Average Assault Map Potential %") + 1, "Average Assault Map Potential % Rank", cowsrank)
        
    elif name == "Control":
        stats.insert(stats.columns.get_loc("oasis_Map Potential % Rank") + 1, "Total Control Played", times_playedtotal)
        stats.insert(stats.columns.get_loc("Total Control Played") + 1, "Average Control Points Earned", earnedtotal)
        stats.insert(stats.columns.get_loc("Average Control Points Earned") + 1, "Average Control Points Lost", losttotal)
        stats.insert(stats.columns.get_loc("Average Control Points Lost") + 1, "Average Control Points Differential", differtotal)
        stats.insert(stats.columns.get_loc("Average Control Points Differential") + 1, "Average Control Points Differential Rank", differrank)
        stats.insert(stats.columns.get_loc("Average Control Points Differential Rank") + 1, "Average Control Win %", winperctotal)
        stats.insert(stats.columns.get_loc("Average Control Win %") + 1, "Average Control Win % Rank", winpercrank)
        stats.insert(stats.columns.get_loc("Average Control Win % Rank") + 1, "Average Control True Win %", trueperctotal)
        stats.insert(stats.columns.get_loc("Average Control True Win %") + 1, "Average Control True Win % Rank", truepercrank)
        stats.insert(stats.columns.get_loc("Average Control True Win % Rank") + 1, "Average Control Map Potential %", cowstotal)
        stats.insert(stats.columns.get_loc("Average Control Map Potential %") + 1, "Average Control Map Potential % Rank", cowsrank)
    
    elif name == "Hybrid":
        stats.insert(stats.columns.get_loc("numbani_Map Potential % Rank") + 1, "Total Hybrid Played", times_playedtotal)
        stats.insert(stats.columns.get_loc("Total Hybrid Played") + 1, "Average Hybrid Points Earned", earnedtotal)
        stats.insert(stats.columns.get_loc("Average Hybrid Points Earned") + 1, "Average Hybrid Points Lost", losttotal)
        stats.insert(stats.columns.get_loc("Average Hybrid Points Lost") + 1, "Average Hybrid Points Differential", differtotal)
        stats.insert(stats.columns.get_loc("Average Hybrid Points Differential") + 1, "Average Hybrid Points Differential Rank", differrank)
        stats.insert(stats.columns.get_loc("Average Hybrid Points Differential Rank") + 1, "Average Hybrid Win %", winperctotal)
        stats.insert(stats.columns.get_loc("Average Hybrid Win %") + 1, "Average Hybrid Win % Rank", winpercrank)
        stats.insert(stats.columns.get_loc("Average Hybrid Win % Rank") + 1, "Average Hybrid True Win %", trueperctotal)
        stats.insert(stats.columns.get_loc("Average Hybrid True Win %") + 1, "Average Hybrid True Win % Rank", truepercrank)
        stats.insert(stats.columns.get_loc("Average Hybrid True Win % Rank") + 1, "Average Hybrid Map Potential %", cowstotal)
        stats.insert(stats.columns.get_loc("Average Hybrid Map Potential %") + 1, "Average Hybrid Map Potential % Rank", cowsrank)
    
    elif name == "Escort":
        stats.insert(stats.columns.get_loc("gibraltar_Map Potential % Rank") + 1, "Total Escort Played", times_playedtotal)
        stats.insert(stats.columns.get_loc("Total Escort Played") + 1, "Average Escort Points Earned", earnedtotal)
        stats.insert(stats.columns.get_loc("Average Escort Points Earned") + 1, "Average Escort Points Lost", losttotal)
        stats.insert(stats.columns.get_loc("Average Escort Points Lost") + 1, "Average Escort Points Differential", differtotal)
        stats.insert(stats.columns.get_loc("Average Escort Points Differential") + 1, "Average Escort Points Differential Rank", differrank)
        stats.insert(stats.columns.get_loc("Average Escort Points Differential Rank") + 1, "Average Escort Win %", winperctotal)
        stats.insert(stats.columns.get_loc("Average Escort Win %") + 1, "Average Escort Win % Rank", winpercrank)
        stats.insert(stats.columns.get_loc("Average Escort Win % Rank") + 1, "Average Escort True Win %", trueperctotal)
        stats.insert(stats.columns.get_loc("Average Escort True Win %") + 1, "Average Escort True Win % Rank", truepercrank)
        stats.insert(stats.columns.get_loc("Average Escort True Win % Rank") + 1, "Average Escort Map Potential %", cowstotal)
        stats.insert(stats.columns.get_loc("Average Escort Map Potential %") + 1, "Average Escort Map Potential % Rank", cowsrank)
        
    else:
        pass
    
sum_rows(assault, "Assault")
sum_rows(control, "Control")
sum_rows(hybrid, "Hybrid")
sum_rows(escort, "Escort")

#converts all data to a csv
stats.to_csv("stats.csv")